In [1]:
import pandas as pd

In [2]:
chunksize = 10**5
results = []

for chunk in pd.read_csv('../data/khyo9nd4xhpobdzx.csv', chunksize=chunksize, dtype={
    'SYM_SUFFIX': str,
    'SYM_ROOT': str,
    'EX': str,
    'TR_SCOND': str
}):
# for chunk in pd.read_csv('../data/khyo9nd4xhpobdzx.csv', chunksize=chunksize):
    chunk['timestamp'] = pd.to_datetime(chunk['DATE'] + ' ' + chunk['TIME_M'])
    
    # chunk.set_index('timestamp', inplace=True)

    # resampled = chunk.resample('1min').PRICE.mean()
    chunk['PRICE'] = pd.to_numeric(chunk['PRICE'], errors='coerce')
    
    chunk = chunk[['timestamp', 'SYM_ROOT', 'PRICE']]

    chunk.set_index('timestamp', inplace=True)

    resampled = (
        chunk
        .groupby('SYM_ROOT')
        .resample('1min')
        .agg({'PRICE': 'mean'})
        .reset_index()
    )

    results.append(resampled)
    # break

/var/folders/6x/z_x1293938b_qn5ylrnb86d40000gn/T/ipykernel_10956/1628410027.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/khyo9nd4xhpobdzx.csv', chunksize=chunksize, dtype={
/var/folders/6x/z_x1293938b_qn5ylrnb86d40000gn/T/ipykernel_10956/1628410027.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/khyo9nd4xhpobdzx.csv', chunksize=chunksize, dtype={
/var/folders/6x/z_x1293938b_qn5ylrnb86d40000gn/T/ipykernel_10956/1628410027.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/khyo9nd4xhpobdzx.csv', chunksize=chunksize, dtype={
/var/folders/6x/z_x1293938b_qn5ylrnb86d40000gn/T/ipykernel_10956/1628410027.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  for ch

In [3]:
final = pd.concat(results)

In [4]:
final = (
    final
    .groupby(['timestamp', 'SYM_ROOT'])['PRICE']
    .mean()
    .reset_index()
)

final = final.rename(columns={'PRICE': 'avg_price'})

In [5]:
final.head()

,timestamp,SYM_ROOT,avg_price
0,2018-01-02 04:00:00,AAPL,169.190909
1,2018-01-02 04:00:00,AMZN,1170.662500
2,2018-01-02 04:00:00,BA,295.000000
3,2018-01-02 04:00:00,C,74.450000
4,2018-01-02 04:00:00,DIS,107.510000


In [6]:
final.tail()

,timestamp,SYM_ROOT,avg_price
23895537,2018-12-31 19:59:00,NVAX,1.76
23895538,2018-12-31 19:59:00,SQ,56.15
23895539,2018-12-31 19:59:00,TSLA,333.30
23895540,2018-12-31 19:59:00,WYNN,98.63
23895541,2018-12-31 19:59:00,XOM,68.19


In [7]:
len(final)

23895542

In [8]:
final.describe()

,timestamp,avg_price
count,23895542,5.347726e+06
mean,2018-07-03 07:25:42.463943424,1.947901e+02
min,2018-01-02 04:00:00,1.000000e-04
25%,2018-04-02 10:40:00,4.646500e+01
50%,2018-07-03 09:14:00,8.175564e+01
75%,2018-10-03 03:02:00,1.627000e+02
max,2018-12-31 19:59:00,2.049861e+03
std,NaN,3.476683e+02


In [9]:
final.to_parquet(
    '../data/taq/',
    engine='pyarrow',
    index=False,
    partition_cols=['SYM_ROOT']  # You can use any column here
)